# Treinando Sentence-BERT com Cosine Similarity Loss

Neste notebook, vamos treinar um modelo **Sentence-BERT (sBERT)** para medir similaridade entre sentenças usando a **Cosine Similarity Loss**.

O objetivo é fazer o modelo aprender a **aproximar embeddings** de sentenças semelhantes e **afastar embeddings** de sentenças diferentes.

## 1. Instalação e importação das bibliotecas

In [1]:
import os
import torch
import pandas as pd
import accelerate, transformers
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from torch.utils.data import DataLoader, Dataset

Se você tiver acesso a uma **GPU** local, irá aparecer o nome abaixo:

In [8]:
print("CUDA disponível:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Nenhuma GPU detectada")

CUDA disponível: True
GPU: NVIDIA GeForce GTX 1650


## 2. Carregar o dataset de treino

O arquivo `data/train.csv` contém pares de sentenças e um valor de similaridade (`similarity_score`) entre 0 e 5.

In [3]:
df = pd.read_csv("../data/train.csv")
df.head()

,question,answer,label
0,"Who is the author of the book , `` The Iron La...",the IRON LADY ; A Biography of Margaret Thatch...,1
1,"Who is the author of the book , `` The Iron La...","in this same revisionist mold , Hugo Young , t...",1
2,"Who is the author of the book , `` The Iron La...","in `` The Iron Lady , '' Young traces the wind...",1
3,"Who is the author of the book , `` The Iron La...",`` He is the very essence of the classless mer...,1
4,"Who is the author of the book , `` The Iron La...","From her father , Young argues , she inherited...",0


## 3. Preparar os exemplos para treino

O Sentence-BERT espera exemplos no formato `InputExample`, contendo dois textos e uma label (score de similaridade).

In [4]:
train_examples = [
    InputExample(
        texts=[row['question'], row['answer']],
        label=float(row['label'])
    )
    for _, row in df.iterrows()
]

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)  # type: ignore
print(f"✅ Total de exemplos de treino: {len(train_examples)}")

✅ Total de exemplos de treino: 4718


## 4. Carregar o modelo sBERT

Usaremos o modelo `all-MiniLM-L12-v2`, que é bem eficiente para tarefas de similaridade semântica.
Obs.: Também pode testar com o modelo `all-MiniLM-L6-v2`

Também ativa o treinamento pela GPU, se estiver disponível.

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('all-MiniLM-L12-v2', device=device)

## 5. Definir a função de perda CosineSimilarityLoss

A função `CosineSimilarityLoss` faz o modelo maximizar a similaridade do cosseno entre embeddings de sentenças semelhantes e minimizar a de sentenças diferentes.

In [6]:
train_loss = losses.CosineSimilarityLoss(model=model)

## 6. Treinar o modelo

In [7]:
output_dir = '../output/sbert-cosine'
os.makedirs(output_dir, exist_ok=True)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=5,               # número de épocas no treinamento
    warmup_steps=100,       # número de passos para aquecimento, evita overfitting
    output_path=output_dir, # diretório para salvar o modelo
    show_progress_bar=True, # mostra barra de progresso
)

print(f"✅ Treinamento concluído! Modelo salvo em: {output_dir}")

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,0.048300
1000,0.022700


✅ Treinamento concluído! Modelo salvo em: ../output/sbert-cosine


## Conclusão

O modelo foi treinado com **Cosine Similarity Loss**, que aprende a prever o grau contínuo de similaridade entre pares de sentenças.

No próximo notebook, vamos **avaliar** esse modelo no conjunto de teste (`test.csv`) e comparar com o `ground_truth.csv`."